In [2]:
import numpy as np
from scipy.optimize import linprog

# Problem parameters
num_facilities = 3
num_customers = 5
num_scenarios = 100

# Fixed costs for opening facilities
fixed_costs = np.array([100000, 120000, 90000])

# Transportation costs from facilities to customers
trans_costs = np.array([
    [10, 15, 20, 25, 30],
    [12, 14, 18, 22, 26],
    [8, 16, 24, 32, 40]
])

# Facility capacities
capacities = np.array([5000, 6000, 4500])

# Generate random demand scenarios
np.random.seed(42)
demand_mean = 1000
demand_std = 200
demand_scenarios = np.random.normal(demand_mean, demand_std, (num_scenarios, num_customers))
demand_scenarios = np.clip(demand_scenarios, 0, None)  # Ensure non-negative demand

# Penalty parameter for the algorithm
rho = 1.0

# Number of iterations
max_iterations = 100

# Convergence tolerance
tolerance = 1e-4

# Initialize decision variables and dual variables
x = np.zeros((num_scenarios, num_facilities, num_customers))
y = np.zeros((num_scenarios, num_facilities))  # Binary decision for opening facilities
lambda_x = np.zeros((num_scenarios, num_facilities, num_customers))
lambda_y = np.zeros((num_scenarios, num_facilities))

# Helper function to solve the subproblem for each scenario
def solve_subproblem(demand, fixed_costs, trans_costs, capacities, lambda_x, lambda_y, rho, s):
    num_facilities, num_customers = trans_costs.shape

    # Objective function coefficients for transportation and facility opening
    c = np.concatenate([trans_costs.flatten(), fixed_costs])
    
    # Inequality constraints (capacity constraints and demand constraints)
    A = np.zeros((num_facilities + num_customers, num_facilities * num_customers + num_facilities))
    b = np.zeros(num_facilities + num_customers)
    
    # Capacity constraints
    for i in range(num_facilities):
        A[i, i*num_customers:(i+1)*num_customers] = 1
        b[i] = capacities[i]
    
    # Demand constraints
    for j in range(num_customers):
        A[num_facilities + j, j::num_customers] = -1
        b[num_facilities + j] = -demand[j]

    # Variable bounds
    bounds = [(0, None) for _ in range(num_facilities * num_customers)] + [(0, 1) for _ in range(num_facilities)]
    
    # Adjust objective function coefficients with dual variables and penalty terms

    adjusted_c = np.concatenate([
        c[:num_facilities * num_customers] + lambda_x.flatten() + rho * x[s].flatten(),
        c[num_facilities * num_customers:] + lambda_y + rho * y[s].flatten()
    ])
    
    # Solve the linear programming problem
    result = linprog(adjusted_c, A_ub=A, b_ub=b, bounds=bounds, method='highs')
    
    # Extract solution
    x_sol = result.x[:num_facilities * num_customers].reshape((num_facilities, num_customers))
    y_sol = result.x[num_facilities * num_customers:]
    
    return x_sol, y_sol

# Progressive Hedging Algorithm
for iteration in range(max_iterations):
    x_old = x.copy()
    y_old = y.copy()
    
    # Solve subproblems for each scenario
    for s in range(num_scenarios):
        x[s], y[s] = solve_subproblem(demand_scenarios[s], fixed_costs, trans_costs, capacities, lambda_x[s], lambda_y[s], rho, s)
    
    # Compute averages
    x_avg = np.mean(x, axis=0)
    y_avg = np.mean(y, axis=0)
    
    # Update dual variables
    for s in range(num_scenarios):
        lambda_x[s] += rho * (x[s] - x_avg)
        lambda_y[s] += rho * (y[s] - y_avg)
    
    # Check for convergence
    if np.max(np.abs(x - x_old)) < tolerance and np.max(np.abs(y - y_old)) < tolerance:
        print(f"Converged in {iteration + 1} iterations")
        break

# Display final decision variables
print("Facility opening decisions (y):")
print(np.round(y_avg))
print("Transportation decisions (x):")
print(np.round(x_avg))


Facility opening decisions (y):
[0. 0. 0.]
Transportation decisions (x):
[[1164.  954. 1027. 1131.  644.]
 [1281. 1218. 1061.  908. 1383.]
 [ 931.  809.  818.  976.  856.]]
